In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
# Load 2010 county GIS data
counties_2010 = gpd.read_file('data/tl_2010_us_county10.shp')
counties_2010 = counties_2010[counties_2010['STATEFP10'].isin([str(i).zfill(2) for i in range(1, 57)])]  # Keep only states and DC
counties_2010 = counties_2010[~counties_2010['STATEFP10'].isin(['02', '15'])]  # Remove Alaska and Hawaii
pd.DataFrame(counties_2010)

,STATEFP10,COUNTYFP10,COUNTYNS10,GEOID10,NAME10,NAMELSAD10,LSAD10,CLASSFP10,MTFCC10,CSAFP10,CBSAFP10,METDIVFP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
2,28,107,00695776,28107,Panola,Panola County,06,H1,G4020,None,None,None,A,1774515519,51767046,+34.3652052,-089.9630654,"POLYGON ((-90.13476 34.22603, -90.13476 34.226..."
3,28,101,00695773,28101,Newton,Newton County,06,H1,G4020,None,None,None,A,1497282694,3879399,+32.4019702,-089.1184115,"POLYGON ((-89.13497 32.57697, -89.13466 32.576..."
4,28,027,00695738,28027,Coahoma,Coahoma County,06,H1,G4020,None,17260,None,A,1430818231,79539470,+34.2286699,-090.6031650,"POLYGON ((-90.59062 33.9869, -90.59473 33.9869..."
5,22,065,00558088,22065,Madison,Madison Parish,15,H1,G4020,None,45260,None,A,1617283455,68538668,+32.3658243,-091.2407290,"MULTIPOLYGON (((-91.03511 32.12035, -91.03621 ..."
6,51,540,01789068,51540,Charlottesville,Charlottesville city,25,C7,G4020,None,16820,None,F,26517362,52974,+38.0376579,-078.4853806,"POLYGON ((-78.47071 38.04872, -78.47033 38.048..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,37,077,01008556,37077,Granville,Granville County,06,H1,G4020,None,None,None,A,1376758990,12765614,+36.2998840,-078.6576338,"POLYGON ((-78.80329 36.17716, -78.80324 36.179..."
3217,37,111,01025836,37111,McDowell,McDowell County,06,H1,G4020,None,None,None,A,1141169175,13956758,+35.6822317,-082.0480292,"POLYGON ((-81.86107 35.69329, -81.86077 35.692..."
3218,27,021,00659456,27021,Cass,Cass County,06,H1,G4020,None,14660,None,A,5235770376,1016739919,+46.9514271,-094.3337731,"POLYGON ((-94.65939 46.8907, -94.65936 46.8920..."
3219,27,057,00659474,27057,Hubbard,Hubbard County,06,H1,G4020,None,None,None,A,2397485927,191360735,+47.0955506,-094.9132898,"POLYGON ((-95.16917 47.15252, -95.16917 47.155..."


Important: Nantucket (FIPS# 25019) not included in CACES dataset.

In [4]:
# Load county PM2.5, NO2, O3, SO2 data from CACES
pm25no2 = pd.read_csv('data/CACES_2017_2019_CONUS.csv')
# Pad fips codes with leading zeros if necessary
pm25no2['fips'] = pm25no2['fips'].apply(lambda x: str(x).zfill(5))
pm25 = pm25no2[pm25no2['pollutant'] == 'pm25']
no2 = pm25no2[pm25no2['pollutant'] == 'no2']
o3 = pm25no2[pm25no2['pollutant'] == 'o3']
so2 = pm25no2[pm25no2['pollutant'] == 'so2']

In [5]:
# Aggregate each pollutant to 2017-2019 average by county
pm25_avg = pm25.groupby(['fips']).agg({'pred_wght': 'mean'}).reset_index()
pm25_avg = pm25_avg.rename(columns={'pred_wght': 'pred_wght_pm25'})
no2_avg = no2.groupby(['fips']).agg({'pred_wght': 'mean'}).reset_index()
no2_avg = no2_avg.rename(columns={'pred_wght': 'pred_wght_no2'})
o3_avg = o3.groupby(['fips']).agg({'pred_wght': 'mean'}).reset_index()
o3_avg = o3_avg.rename(columns={'pred_wght': 'pred_wght_o3'})
so2_avg = so2.groupby(['fips']).agg({'pred_wght': 'mean'}).reset_index()
so2_avg = so2_avg.rename(columns={'pred_wght': 'pred_wght_so2'})

In [6]:
# Preview of merged PM2.5, NO2, O3, SO2 averages
merged_pollutants = pd.merge(pm25_avg, no2_avg, on='fips', how='inner')
merged_pollutants = pd.merge(merged_pollutants, o3_avg, on='fips', how='inner')
merged_pollutants = pd.merge(merged_pollutants, so2_avg, on='fips', how='inner')
merged_pollutants

,fips,pred_wght_pm25,pred_wght_no2,pred_wght_o3,pred_wght_so2
0,01001,7.205508,2.774375,37.618847,0.707542
1,01003,6.873977,2.213090,38.356880,0.436102
2,01005,7.222006,1.605340,36.587833,0.686061
3,01007,7.168470,1.722987,37.928522,0.721835
4,01009,7.288260,2.060355,41.278875,0.638830
...,...,...,...,...,...
3103,56037,5.035663,5.204038,49.158762,1.176892
3104,56039,4.016307,2.393979,46.595220,0.261468
3105,56041,5.662810,4.473194,51.062664,0.988933
3106,56043,5.148899,3.446874,45.580613,0.940092


In [ ]:
# Load 2021 PLACES data
places_2021 = gpd.read_file('data/places_2021.csv')[['StateAbbr', 'CountyName', 'CountyFIPS', 'CASTHMA_AdjPrev', 
                                                     'CASTHMA_Adj95CI', 'CSMOKING_AdjPrev', 'TotalPopulation', 'Geolocation']]
# Load 2020 PLACES data
places_2020 = gpd.read_file('data/places_2020.csv')[['StateAbbr', 'CountyName', 'CountyFIPS', 'CASTHMA_AdjPrev', 
                                                     'CASTHMA_Adj95CI', 'CSMOKING_AdjPrev', 'TotalPopulation', 'Geolocation']]
# Narrow to CONUS
places_2021 = places_2021[~places_2021['StateAbbr'].isin(['AK', 'HI', 'PR', 'VI', 'GU', 'MP', 'AS'])]
# Fix Oglala Lakota County FIPS code
places_2021.loc[places_2021['CountyFIPS'] == '46102', 'CountyFIPS'] = '46113'
# Since PLACES 2021 is missing New Jersey data, we will use 2020 data for it
places_2020_nj = places_2020[places_2020['StateAbbr'] == 'NJ']
places_2021 = pd.concat([places_2021[places_2021['StateAbbr'] != 'NJ'], places_2020_nj])
places_2021

,StateAbbr,CountyName,CountyFIPS,CASTHMA_AdjPrev,CASTHMA_Adj95CI,CSMOKING_AdjPrev,TotalPopulation,Geolocation
1,OK,Jackson,40065,9.7,"( 9.3, 10.2)",17.4,"24,530",POINT (-99.41467445 34.58811786)
2,IL,Lawrence,17101,9.5,"( 9.0, 10.1)",21.7,"15,678",POINT (-87.72684251 38.7203392)
3,KS,Barton,20009,9.7,"( 9.2, 10.2)",18.7,"25,779",POINT (-98.75632156 38.478934)
5,OK,Stephens,40137,10.5,"( 9.9, 11.1)",21.8,"43,143",POINT (-97.85149263 34.485439)
7,WV,Mingo,54059,12.2,"(11.4, 13.2)",29.5,"23,424",POINT (-82.13487448 37.7266469)
...,...,...,...,...,...,...,...,...
1790,NJ,Salem,34033,10.3,"( 9.8, 10.9)",20.3,"62,607",POINT (-75.34885125 39.58713198)
1791,NJ,Somerset,34035,8.3,"( 7.9, 8.7)",11.9,"331,164",POINT (-74.61618309 40.56359097)
1792,NJ,Sussex,34037,8.9,"( 8.4, 9.4)",14.8,"140,799",POINT (-74.69015804 41.13929435)
1793,NJ,Union,34039,9.5,"( 9.1, 10.0)",15.2,"558,067",POINT (-74.30884526 40.66048019)


In [8]:
# Join pollutant averages to PLACES data
places_2021_poll = places_2021.merge(merged_pollutants, left_on='CountyFIPS', right_on='fips', how='inner')
places_2021_poll

,StateAbbr,CountyName,CountyFIPS,CASTHMA_AdjPrev,CASTHMA_Adj95CI,CSMOKING_AdjPrev,TotalPopulation,Geolocation,fips,pred_wght_pm25,pred_wght_no2,pred_wght_o3,pred_wght_so2
0,OK,Jackson,40065,9.7,"( 9.3, 10.2)",17.4,"24,530",POINT (-99.41467445 34.58811786),40065,7.276856,3.713581,47.733319,1.062317
1,IL,Lawrence,17101,9.5,"( 9.0, 10.1)",21.7,"15,678",POINT (-87.72684251 38.7203392),17101,8.535631,4.100099,43.994110,0.976170
2,KS,Barton,20009,9.7,"( 9.2, 10.2)",18.7,"25,779",POINT (-98.75632156 38.478934),20009,7.053983,3.778923,43.866222,0.993359
3,OK,Stephens,40137,10.5,"( 9.9, 11.1)",21.8,"43,143",POINT (-97.85149263 34.485439),40137,6.857535,2.808665,46.527534,0.810107
4,WV,Mingo,54059,12.2,"(11.4, 13.2)",29.5,"23,424",POINT (-82.13487448 37.7266469),54059,5.357986,2.197716,39.290778,1.523081
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3102,NJ,Salem,34033,10.3,"( 9.8, 10.9)",20.3,"62,607",POINT (-75.34885125 39.58713198),34033,6.915251,4.384872,42.741907,0.510655
3103,NJ,Somerset,34035,8.3,"( 7.9, 8.7)",11.9,"331,164",POINT (-74.61618309 40.56359097),34035,7.191377,6.700827,41.674132,0.578061
3104,NJ,Sussex,34037,8.9,"( 8.4, 9.4)",14.8,"140,799",POINT (-74.69015804 41.13929435),34037,5.406745,3.392481,38.198758,0.635989
3105,NJ,Union,34039,9.5,"( 9.1, 10.0)",15.2,"558,067",POINT (-74.30884526 40.66048019),34039,7.882319,11.928031,40.447981,0.464380


In [13]:
# Create 33/66/100 risk categories based on tertiles of asthma prevalence
def categorize_risk(prevalence):

    if prevalence < places_2021_poll['CASTHMA_AdjPrev'].quantile(0.33):
        return 'Low'
    elif prevalence < places_2021_poll['CASTHMA_AdjPrev'].quantile(0.66):
        return 'Medium'
    else:
        return 'High'
    
places_2021_poll['CASTHMA_AdjPrev'] = pd.to_numeric(places_2021_poll['CASTHMA_AdjPrev'], errors='coerce')
places_2021_poll['CASTHMA_Category'] = places_2021_poll['CASTHMA_AdjPrev'].apply(categorize_risk)

In [14]:
places_2021_poll = places_2021_poll[['CountyFIPS', 'CountyName', 'StateAbbr', 'CASTHMA_AdjPrev', 'CASTHMA_Category',
                                   'pred_wght_pm25', 'pred_wght_no2', 'pred_wght_o3', 'pred_wght_so2',
                                   'CSMOKING_AdjPrev', 'Geolocation']]

places_2021_poll

,CountyFIPS,CountyName,StateAbbr,CASTHMA_AdjPrev,CASTHMA_Category,pred_wght_pm25,pred_wght_no2,pred_wght_o3,pred_wght_so2,CSMOKING_AdjPrev,Geolocation
0,40065,Jackson,OK,9.7,Medium,7.276856,3.713581,47.733319,1.062317,17.4,POINT (-99.41467445 34.58811786)
1,17101,Lawrence,IL,9.5,Medium,8.535631,4.100099,43.994110,0.976170,21.7,POINT (-87.72684251 38.7203392)
2,20009,Barton,KS,9.7,Medium,7.053983,3.778923,43.866222,0.993359,18.7,POINT (-98.75632156 38.478934)
3,40137,Stephens,OK,10.5,High,6.857535,2.808665,46.527534,0.810107,21.8,POINT (-97.85149263 34.485439)
4,54059,Mingo,WV,12.2,High,5.357986,2.197716,39.290778,1.523081,29.5,POINT (-82.13487448 37.7266469)
...,...,...,...,...,...,...,...,...,...,...,...
3102,34033,Salem,NJ,10.3,High,6.915251,4.384872,42.741907,0.510655,20.3,POINT (-75.34885125 39.58713198)
3103,34035,Somerset,NJ,8.3,Low,7.191377,6.700827,41.674132,0.578061,11.9,POINT (-74.61618309 40.56359097)
3104,34037,Sussex,NJ,8.9,Low,5.406745,3.392481,38.198758,0.635989,14.8,POINT (-74.69015804 41.13929435)
3105,34039,Union,NJ,9.5,Medium,7.882319,11.928031,40.447981,0.464380,15.2,POINT (-74.30884526 40.66048019)


In [ ]:
places_2021_poll.to_csv('data/asthma_pollutants.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict
from sklearn.metrics import confusion_matrix, f1_score

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import tree

from imblearn.over_sampling import RandomOverSampler

In [ ]:
file_path = "data/asthma_pollutants.csv"

df = pd.read_csv(file_path)

In [ ]:
# Prepare features and target variable
feature_cols = ['pred_wght_pm25', 'pred_wght_no2', 'pred_wght_o3', 'pred_wght_so2', 'CSMOKING_AdjPrev']
X = df[feature_cols]
# Turn asthma prevalence into low/medium/high categories
# places_2021_poll['CASTHMA_AdjPrev'] = places_2021_poll['CASTHMA_AdjPrev'].astype(float)
# places_2021_poll['CASTHMA_Category'] = pd.qcut(places_2021_poll['CASTHMA_AdjPrev'], q=3, labels=['Low', 'Medium', 'High'])
mapping = {"Low": 0, "Medium": 1, "High": 2}
# df['CASTHMA_Category_Num'] = df['CASTHMA_Category'].map(mapping)
y = df['CASTHMA_Category'].map(mapping)
percentages = y.value_counts(normalize=True) * 100
print(percentages)
target = 'CASTHMA_Category'

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=6242, stratify=y)

# ros = RandomOverSampler(random_state=42)
# X_resampled, y_resampled = ros.fit_resample(X_train, y_train)


In [ ]:
# Initialize RF classifier
classifiers = [tree.DecisionTreeRegressor(), tree.DecisionTreeClassifier(), RandomForestClassifier(n_estimators=1008, random_state=6242), KNeighborsClassifier(n_neighbors=40), AdaBoostClassifier(n_estimators=2000), SGDClassifier(loss="hinge", penalty="l2", max_iter=2000), GradientBoostingClassifier(n_estimators=1000, learning_rate=0.4, max_depth=10), BaggingClassifier(), MLPClassifier(hidden_layer_sizes=(15, 8), max_iter=2000), LogisticRegression(max_iter=1000)]
voting_cl = voting_cl = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(n_estimators=1008, random_state=6242)),
        ('knn', KNeighborsClassifier(n_neighbors=40)),
        ('gb', GradientBoostingClassifier(n_estimators=1000, learning_rate=0.4, max_depth=10)),
    ],
    voting='hard'
)
classifiers.append(voting_cl)

def run_classifier(classifier_input, output_df):
  classifier = classifier_input
  # Train the model
  classifier.fit(X_train, y_train)
  # Evaluate the model
  cv_results = cross_validate(classifier, X, y, cv=5, scoring=['accuracy', 'f1_macro', 'f1_micro', 'f1_weighted'])

  accuracy_scores = cv_results["test_accuracy"]
  weighted_f1_scores = cv_results["test_f1_weighted"]
  # macro_f1_scores = cv_results["test_f1_macro"]
  micro_f1_scores = cv_results["test_f1_micro"]

  print(f"The mean cross validation accuracy is: {accuracy_scores.mean():.1%} +- {accuracy_scores.std():.1%}")
  print(f"The mean cross validation f1 is: {micro_f1_scores.mean():.1%} +- {micro_f1_scores.std():.1%}")
  print(f"The mean cross validation f1 is: {weighted_f1_scores.mean():.1%} +- {weighted_f1_scores.std():.1%}")

  acc_mean = cv_results["test_accuracy"].mean()
  acc_std = cv_results["test_accuracy"].std()
  f1_mean = cv_results["test_f1_weighted"].mean()
  f1_std = cv_results["test_f1_weighted"].std()

  output_df = pd.concat([
      output_df,
      pd.DataFrame([{
          "classifier": classifier.__class__.__name__,
          "accuracy_mean": acc_mean,
          "accuracy_std": acc_std,
          "f1_weighted_mean": f1_mean,
          "f1_weighted_std": f1_std
      }])
  ], ignore_index=True)

  y_pred = cross_val_predict(classifier, X_test, y_test, cv=5)

  matrix = confusion_matrix(y_test, y_pred, labels=[2,1,0])
  print(matrix)

  # print(cv_results)

  return output_df

In [ ]:
# classifier = MLPClassifier(hidden_layer_sizes=(10, 4), max_iter=2000)
output_df = pd.DataFrame(columns=[
    "classifier",
    "accuracy_mean",
    "accuracy_std",
    "f1_weighted_mean",
    "f1_weighted_std"
])
for classifier in classifiers:
  print(classifier)
  output_df= run_classifier(classifier, output_df)

In [ ]:
out_df = pd.DataFrame(columns=[
    "classifier",
    "accuracy_mean",
    "accuracy_std",
    "f1_weighted_mean",
    "f1_weighted_std"
])
classifier = voting_cl = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(n_estimators=1008, random_state=6242)),
        ('knn', KNeighborsClassifier(n_neighbors=40)),
        ('gb', GradientBoostingClassifier(n_estimators=1000, learning_rate=0.4, max_depth=10)),
    ],
    voting='hard'
)

out_df= run_classifier(classifier, output_df)

In [ ]:
feature_cols = ['pred_wght_pm25', 'pred_wght_no2', 'pred_wght_o3', 'pred_wght_so2', 'CSMOKING_AdjPrev']
# X = df[feature_cols]
# y = df['CASTHMA_Category'].map(mapping)

classifier = voting_cl = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(n_estimators=1008, random_state=6242)),
        ('knn', KNeighborsClassifier(n_neighbors=40)),
        ('gb', GradientBoostingClassifier(n_estimators=1000, learning_rate=0.4, max_depth=10)),
    ],
    voting='hard'
)
classifier.fit(X, y)
multipliers = [0.9, 0.8, 0.7, 0.6, 0.5]

results = []
for col in feature_cols:
  for multiplier in multipliers:
    temp_x = X.copy()
    temp_x[feature_cols] = temp_x[feature_cols] * multiplier
    y_pred = cross_val_predict(classifier, temp_x, y, cv=5)

    temp_df = pd.DataFrame({
        "Geolocation": df["Geolocation"],
        "multiplier": [multiplier] * len(y_pred),
        "feature multiplied": [col] * len(y_pred),
        "prediction": y_pred,
        "CountyFIPS": df["CountyFIPS"]
    })
    feature = temp_df["feature multiplied"].iloc[0]
    multiplier = temp_df["multiplier"].iloc[0]

    filename = f"data/what-if/predictions_{feature}_{multiplier}.csv"

    temp_df.to_csv(filename, index=False)
    print(filename)
    print("completed")


In [ ]:
for pred_df in results:
  feature = temp_df["feature multiplied"].iloc[0]
  multiplier = temp_df["multiplier"].iloc[0]

  filename = f"data/what-if/predictions_{feature}_{multiplier}.csv"

  temp_df.to_csv(filename, index=False)
  print(filename)
  print("completed")


In [ ]:
print(output_df)